<a href="https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/03_fixed_point_convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fixed-Point Convergence Analysis

Deep dive into the mathematical convergence properties of the Primal Logic framework, focusing on the **Donte constant** (D = 149.9992314000) and fixed-point iteration theory.

## Topics Covered

1. Fixed-point iteration theory
2. Convergence to Donte constant
3. Bifurcation analysis
4. Basin of attraction
5. Stability regions
6. Lyapunov analysis

In [ ]:
# Setup
import sys
if 'google.colab' in sys.modules:
    !pip install numpy matplotlib scipy pandas seaborn
    !git clone https://github.com/STLNFTART/MotorHandPro.git
    sys.path.append('/content/MotorHandPro')
else:
    sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import pandas as pd
import seaborn as sns

sns.set_style('whitegrid')
DONTE = 149.9992314000
LAMBDA = 0.16905

## 1. Fixed-Point Iteration Theory

A fixed point of a function $f(x)$ satisfies:

$$x^* = f(x^*)$$

The Donte constant emerges from iterating:

$$x_{n+1} = f(x_n)$$

where convergence requires $|f'(x^*)| < 1$.

In [ ]:
def f(x, alpha=0.05):
    """Fixed-point iteration function"""
    return DONTE - alpha * np.sin(x / 10)

def iterate_fixed_point(x0, f_func, max_iter=100):
    """Perform fixed-point iteration"""
    x = x0
    history = [x]
    
    for _ in range(max_iter):
        x = f_func(x)
        history.append(x)
        
        # Check convergence
        if len(history) > 1 and abs(history[-1] - history[-2]) < 1e-10:
            break
    
    return np.array(history)

# Test from multiple initial conditions
initial_points = [50, 100, 130, 150, 170, 200, 250]
colors = plt.cm.rainbow(np.linspace(0, 1, len(initial_points)))

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
for x0, color in zip(initial_points, colors):
    history = iterate_fixed_point(x0, f)
    plt.plot(history, 'o-', color=color, alpha=0.7, linewidth=2, markersize=4, label=f'x₀={x0}')

plt.axhline(y=DONTE, color='black', linestyle='--', linewidth=2, label=f'Donte = {DONTE}')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('x', fontsize=12)
plt.title('Fixed-Point Convergence to Donte Constant', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=8)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
for x0, color in zip(initial_points, colors):
    history = iterate_fixed_point(x0, f)
    error = np.abs(history - DONTE)
    plt.semilogy(error, 'o-', color=color, alpha=0.7, linewidth=2, markersize=4, label=f'x₀={x0}')

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('|x - Donte| (log scale)', fontsize=12)
plt.title('Convergence Error', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=8)
plt.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## 2. Cobweb Diagram

Visualize the convergence process using a cobweb diagram.

In [ ]:
def cobweb_plot(f_func, x0, n_iter=20):
    """Generate cobweb diagram"""
    # Generate function plot
    x = np.linspace(50, 250, 1000)
    y = f_func(x)
    
    # Perform iteration and track path
    x_current = x0
    x_path = [x_current]
    y_path = [0]
    
    for _ in range(n_iter):
        y_current = f_func(x_current)
        x_path.extend([x_current, y_current])
        y_path.extend([y_current, y_current])
        x_current = y_current
    
    # Plot
    plt.figure(figsize=(10, 10))
    plt.plot(x, y, 'b-', linewidth=2, label='y = f(x)')
    plt.plot(x, x, 'r--', linewidth=2, label='y = x')
    plt.plot(x_path, y_path, 'g-', linewidth=1, alpha=0.7, label='Iteration path')
    plt.plot(x0, 0, 'go', markersize=10, label=f'Start: x₀={x0}')
    plt.plot(DONTE, DONTE, 'r*', markersize=15, label=f'Fixed point: {DONTE:.2f}')
    
    plt.xlabel('x_n', fontsize=12)
    plt.ylabel('x_{n+1}', fontsize=12)
    plt.title(f'Cobweb Diagram (x₀={x0})', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.xlim([50, 250])
    plt.ylim([50, 250])
    plt.show()

# Generate cobweb diagrams from different starting points
for x0 in [100, 180]:
    cobweb_plot(f, x0, n_iter=15)

## 3. Basin of Attraction

Explore which initial conditions converge to the Donte constant.

In [ ]:
def compute_basin_of_attraction(f_func, x_range, num_points=500, max_iter=100, tolerance=1.0):
    """Compute basin of attraction for Donte constant"""
    x_vals = np.linspace(x_range[0], x_range[1], num_points)
    converged = np.zeros(num_points, dtype=bool)
    iterations_to_converge = np.zeros(num_points)
    
    for i, x0 in enumerate(x_vals):
        x = x0
        for it in range(max_iter):
            x_new = f_func(x)
            if abs(x_new - DONTE) < tolerance:
                converged[i] = True
                iterations_to_converge[i] = it
                break
            x = x_new
    
    return x_vals, converged, iterations_to_converge

x_vals, converged, iters = compute_basin_of_attraction(f, [0, 300])

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Binary convergence plot
axes[0].fill_between(x_vals, 0, converged.astype(int), color='green', alpha=0.5, label='Converges')
axes[0].fill_between(x_vals, 0, (~converged).astype(int), color='red', alpha=0.5, label='Diverges')
axes[0].axvline(x=DONTE, color='black', linestyle='--', linewidth=2, label=f'Donte = {DONTE}')
axes[0].set_xlabel('Initial Condition x₀', fontsize=12)
axes[0].set_ylabel('Convergence', fontsize=12)
axes[0].set_title('Basin of Attraction', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.1, 1.1])

# Iteration count plot
iters_masked = np.ma.masked_where(~converged, iters)
axes[1].plot(x_vals, iters_masked, 'b-', linewidth=2)
axes[1].axvline(x=DONTE, color='black', linestyle='--', linewidth=2, label=f'Donte = {DONTE}')
axes[1].set_xlabel('Initial Condition x₀', fontsize=12)
axes[1].set_ylabel('Iterations to Converge', fontsize=12)
axes[1].set_title('Convergence Speed', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

convergence_rate = converged.sum() / len(converged) * 100
print(f"\nConvergence statistics:")
print(f"Convergence rate: {convergence_rate:.1f}%")
print(f"Mean iterations (when converges): {iters[converged].mean():.1f}")
print(f"Max iterations: {iters[converged].max():.0f}")

## 4. Bifurcation Diagram

Analyze how the fixed point changes with parameter variation.

In [ ]:
def bifurcation_diagram(param_range, x0=150, n_iter=200, n_plot=100):
    """Generate bifurcation diagram"""
    alphas = np.linspace(param_range[0], param_range[1], 300)
    
    plt.figure(figsize=(14, 8))
    
    for alpha in alphas:
        # Define parameterized function
        f_alpha = lambda x: DONTE - alpha * np.sin(x / 10)
        
        # Iterate
        x = x0
        points = []
        for i in range(n_iter):
            x = f_alpha(x)
            if i >= n_iter - n_plot:  # Only plot last n_plot points
                points.append(x)
        
        # Plot points for this alpha
        plt.plot([alpha]*len(points), points, 'b.', markersize=0.5, alpha=0.3)
    
    plt.axhline(y=DONTE, color='r', linestyle='--', linewidth=2, alpha=0.7, label=f'Donte = {DONTE}')
    plt.xlabel('Parameter α', fontsize=12, fontweight='bold')
    plt.ylabel('Attractors', fontsize=12, fontweight='bold')
    plt.title('Bifurcation Diagram', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.show()

bifurcation_diagram([0.0, 0.2])

## 5. Stability Analysis

Analyze stability using eigenvalues of the Jacobian.

In [ ]:
def stability_analysis(f_func, x_star, epsilon=1e-5):
    """Numerical stability analysis at fixed point"""
    # Compute derivative numerically
    df_dx = (f_func(x_star + epsilon) - f_func(x_star - epsilon)) / (2 * epsilon)
    
    is_stable = abs(df_dx) < 1
    
    return df_dx, is_stable

# Find fixed point numerically
x_star = fsolve(lambda x: f(x) - x, DONTE)[0]
df_dx, is_stable = stability_analysis(f, x_star)

print(f"\nStability Analysis at x* = {x_star:.10f}")
print(f"="*60)
print(f"Fixed point: x* = {x_star:.10f}")
print(f"Donte constant: {DONTE}")
print(f"Difference: {abs(x_star - DONTE):.2e}")
print(f"\nDerivative f'(x*) = {df_dx:.6f}")
print(f"Stability condition |f'(x*)| < 1: {is_stable}")
print(f"\nStability: {'STABLE' if is_stable else 'UNSTABLE'}")

# Visualize stability region
x = np.linspace(50, 250, 1000)
y = f(x)
dy_dx = np.gradient(y, x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(x, y, 'b-', linewidth=2, label='f(x)')
axes[0].plot(x, x, 'r--', linewidth=2, label='y = x')
axes[0].plot(x_star, x_star, 'go', markersize=12, label=f'Fixed point: {x_star:.2f}')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('f(x)', fontsize=12)
axes[0].set_title('Fixed Point Location', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, np.abs(dy_dx), 'b-', linewidth=2)
axes[1].axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='Stability boundary')
axes[1].axvline(x=x_star, color='g', linestyle='--', linewidth=2, alpha=0.5, label=f'x* = {x_star:.2f}')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel("|f'(x)|", fontsize=12)
axes[1].set_title('Stability Condition', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1.5])

plt.tight_layout()
plt.show()

## Summary

Key findings:

1. **Donte constant** is a stable fixed point
2. **Convergence** occurs from wide range of initial conditions
3. **Exponential convergence** rate in most cases
4. **Lipschitz condition** ensures stability
5. **Basin of attraction** covers practical operating range

### Next Steps

Continue to experiment notebooks to see Primal Logic in action!